In [5]:
import gradio as gr
import requests
import json

url = "https://client.camb.ai/apis/tts"



# print(response.text)

In [7]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, StringPromptTemplate, FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

_ = load_dotenv()

In [8]:
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
camb_ai_key = os.getenv("CAMB_AI_KEY")

In [4]:
def cambai_getvoices():
    API_KEY = camb_ai_key
    url = "https://client.camb.ai/apis/list_voices"
    headers = {"x-api-key": API_KEY}
    response = requests.get(url, headers=headers)
    data = response.json()
    
    for voice in data["payload"]:
        print(f"{voice['voice_name']}; {voice['id']}")

In [18]:
def get_task_id(input_str: str): 
    url = "https://client.camb.ai/apis/tts"
    payload = {
        "text": input_str,
        "voice_id": 8745,
        "language": 4,
        "gender": 0,
        "age": 123
    }
    headers = {
        "x-api-key": camb_ai_key,
        "Content-Type": "application/json"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    
    return eval(response.text)

 # Get poll status
def get_poll_status(task_id: str):
    # Job Status
    url = f"https://client.camb.ai/apis/tts/{task_id}"

    headers = {"x-api-key": camb_ai_key}

    response = requests.request("GET", url, headers=headers)
    
    return response.text

In [19]:
task_id = get_task_id("أنا أتعلم اللغة العربية بشكل جيد")['task_id']

In [22]:
poll_status = get_poll_status(task_id)

In [25]:
poll_status = eval(poll_status)

In [26]:
tts_result = requests.get(
    f"https://client.camb.ai/apis/tts_result/{poll_status['run_id']}",
    headers={"x-api-key": camb_ai_key}, stream=True
)
with open("saved_stream_arabic.wav", "wb") as f:
    for chunk in tts_result.iter_content(chunk_size=1024):
        f.write(chunk)

In [42]:
import time

In [10]:
def clean_input(raw_input: str):
    openai_model = ChatOpenAI(temperature=0, api_key=openai_api_key, model='gpt-3.5-turbo')
    SYSTEM_PROMPT = """You are an advanced language model trained to rephrase sentences while removing inappropriate, offensive, or explicit content. Your task is to take any input sentence, no matter how vulgar or offensive, and rewrite it in a polite, family-friendly manner without losing the core meaning."""
    human = "{text}"
    prompt = ChatPromptTemplate.from_messages([("system", SYSTEM_PROMPT), ("human", human)])

    chain = prompt | openai_model | StrOutputParser()
    paraphrased_sentence = chain.invoke({"text": raw_input})
    return paraphrased_sentence

In [13]:
def cambAI_TTS(input_str: str):
    print(f"Input text: {input_str}")
    paraphrased_sentence = clean_input(input_str)
    print(f"Paraphrased text: {paraphrased_sentence}")
    task_id = get_task_id(paraphrased_sentence)['task_id']
    time.sleep(10)
    poll_status = get_poll_status(task_id) 
    poll_status = eval(poll_status)
    if poll_status['status'] == 'SUCCESS':
        tts_result = requests.get(
            f"https://client.camb.ai/apis/tts_result/{poll_status['run_id']}",
            headers={"x-api-key": camb_ai_key}, stream=True
        )
        with open("saved_stream.wav", "wb") as f:
            for chunk in tts_result.iter_content(chunk_size=1024):
                f.write(chunk)

    if poll_status['status'] == "FAILED":
        print('Job failed!')
        return None
    
    return "saved_stream.wav"

In [16]:
app = gr.Interface(fn=cambAI_TTS,
                    inputs='textbox',
                    outputs='audio')

app.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [98]:
# Araclip demo 
with gr.Blocks() as demo_araclip:

    gr.Markdown("## Input parameters")
    
    txt = gr.Textbox(label="Text Query (Caption)")
    
    out_voice = gr.Audio(label="Audio Output")

    with gr.Row():
        btn = gr.Button("Start", scale=1)
    

    btn.click(cambAI_TTS, inputs=[txt], outputs=[out_voice])

# Group the demos in a TabbedInterface 
with gr.Blocks() as demo:

    gr.TabbedInterface([demo_araclip], ["Our Model"])

if __name__ == "__main__":
    
    demo.launch()

## Case 2

In [1]:
import mishkal.tashkeel

vocalizer = mishkal.tashkeel.TashkeelClass()

for i in range(5):
    text = u"بسم الله الرحمن الرحيم"
    print(vocalizer.tashkeel(text))

 بِسْمِ اللهِ الرَّحْمَنِ الرَّحِيمِ
 بِسْمِ اللهِ الرَّحْمَنِ الرَّحِيمِ
 بِسْمِ اللهِ الرَّحْمَنِ الرَّحِيمِ
 بِسْمِ اللهِ الرَّحْمَنِ الرَّحِيمِ
 بِسْمِ اللهِ الرَّحْمَنِ الرَّحِيمِ


In [14]:
def cambAI_TTS(input_str: str):
    print(f"Input text: {input_str}")
    paraphrased_sentence = vocalizer.tashkeel(input_str)
    print(f"Paraphrased text: {paraphrased_sentence}")
    task_id = get_task_id(paraphrased_sentence)['task_id']
    time.sleep(10)
    poll_status = get_poll_status(task_id) 
    poll_status = eval(poll_status)
    if poll_status['status'] == 'SUCCESS':
        tts_result = requests.get(
            f"https://client.camb.ai/apis/tts_result/{poll_status['run_id']}",
            headers={"x-api-key": camb_ai_key}, stream=True
        )
        with open("saved_stream.wav", "wb") as f:
            for chunk in tts_result.iter_content(chunk_size=1024):
                f.write(chunk)

    if poll_status['status'] == "FAILED":
        print('Job failed!')
        return None
    
    return "saved_stream.wav"

In [15]:
# Araclip demo 
with gr.Blocks() as demo_araclip:

    gr.Markdown("## Input parameters")
    
    txt = gr.Textbox(label="Text Query (Caption)")
    
    out_voice = gr.Audio(label="Audio Output")

    with gr.Row():
        btn = gr.Button("Start", scale=1)
    

    btn.click(cambAI_TTS, inputs=[txt], outputs=[out_voice])

# Group the demos in a TabbedInterface 
with gr.Blocks() as demo:

    gr.TabbedInterface([demo_araclip], ["Our Model"])

if __name__ == "__main__":
    
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [27]:
app = gr.Interface(
    fn=cambAI_TTS,
    inputs=[
        gr.Textbox(label='Enter your offensive sentence'),
        gr.Dropdown(
           choices=["en", "ar"], label='Language'
        )
    ],
  outputs=[
      gr.Audio(label='TTS output from CAMB.AI'),
      gr.Textbox(label='Cleaned output from LLM')
  ]
)

app.launch()

/Users/isham993/mambaforge/envs/gpu_torch/lib/python3.11/site-packages/gradio/utils.py:977: UserWarning: Expected 1 arguments for function <function cambAI_TTS at 0x1765d8f40>, received 2.
  warnings.warn(
/Users/isham993/mambaforge/envs/gpu_torch/lib/python3.11/site-packages/gradio/utils.py:985: UserWarning: Expected maximum 1 arguments for function <function cambAI_TTS at 0x1765d8f40>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
